In [1]:
import pandas as pd
import geopandas as gpd
import os
import ast
import numpy as np
import math

pd.set_option('display.max_columns', None)
os.chdir(r"C:\Users\LMENENDEZ\GitHub\MultiModX")
print(os.getcwd())

C:\Users\LMENENDEZ\GitHub\MultiModX


In [2]:
%load_ext autoreload

In [3]:
%autoreload
from script.trips_format import *

## 1. Download and rescale trips

In [4]:
all_trips = pd.read_csv(
    r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\Matrices MITMA\with_archetypes\20220922_28_trip_matrix_arc_pt_processed.csv.gz",
    compression="gzip",
    sep="|"
)

In [5]:
trips = all_trips[all_trips["date"] == 20220923].reset_index(drop=True).rename(columns={"origin_nut": "origin", "destination_nut": "destination"})

In [6]:
#associates each airport to the corresponding new NUTS
airports_to_NUTS={"airport_LPA":("ES705","Gran Canaria"),
                 "airport_FUE":("ES704","Fuerteventura"),
                 "airport_ACE":("ES708","Lanzarote"),
                 "airport_TFS":("ES709","Tenerife"),
                 "airport_TFN":("ES709","Tenerife"),
                 "airport_GMZ":("ES709","Tenerife"),
                 "airport_SPC":("ES707","La Palma"),
                 "airport_VDE":("ES703","El Hierro"),
                 "airport_PMI":("ES532","Mallorca"),
                 "airport_IBZ":("ES531","Eivissa i Formentera"),
                 "airport_MAH":("ES533","Menorca")}

In [7]:
trips=format_trips(trips, airports_to_NUTS)

17 columns were removed


In [8]:
# remove cercanías
trips=trips[~(((trips["origin"]=="ES424")&(trips["destination"]=="ES300"))|((trips["origin"]=="ES300")&(trips["destination"]=="ES424")))]

In [9]:
# location of "ALL" train stops given by UiC
# However this list is still incomplete
stops_loc=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\rail_info\stops.txt").astype(str) # everything is a string here to match other formatting
stops_loc["stop_id"] = stops_loc["stop_id"].apply(lambda x: "00" + x) #to make they start with 00

In [10]:
trips.loc[:,"weird_stations"] = trips["node_sequence_reduced"].apply(
    lambda x: find_weird_stations(x, stops_loc))

In [11]:
unique_weird_stations=get_weird_stations(trips["weird_stations"])

In [12]:
MobA_stations_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\datos moba\train_stations\train_stations.shp")

In [13]:
# identifies all the un-localisable stations
nowhere_stations=set(unique_weird_stations)-set(MobA_stations_coord["ID"])
print(f"there are {len(nowhere_stations)} stations that are not in the data provided by MobA but appear in the trips dataframe")

there are 205 stations that are not in the data provided by MobA but appear in the trips dataframe


In [14]:
trips = trips[~trips["node_sequence_reduced"].apply(lambda x: any(station in x for station in nowhere_stations))]

In [15]:
trips_abroad=trips[(trips["origin"]=="abroad")|(trips["destination"]=="abroad")]

In [16]:
# creates a dictionary of mcc code to country acronym
international_codes=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\countries mcc\mcc_to_nationality.txt", sep="|")
mcc_to_country=international_codes.set_index("mcc")["country"].to_dict()

In [17]:
trips_abroad=format_trips_abroad(trips_abroad,mcc_to_country)

In [18]:
coeffs_incoming=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\demand\incoming_trips_coefficients.csv")

In [19]:
coeffs_outgoing=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\demand\outgoing_trips_coefficients.csv")

In [20]:
#we have slightly different countries in coeffs_incoming and coeffs_outgoing
countries_incoming=set(coeffs_incoming["origin"])

countries_outgoing=set(coeffs_outgoing["destination"])

countries_selected=set.union(countries_outgoing,countries_incoming)-(countries_outgoing-countries_incoming)-(countries_incoming-countries_outgoing)

In [21]:
coeffs_incoming=coeffs_incoming[coeffs_incoming["origin"].isin(countries_selected)]
coeffs_outgoing=coeffs_outgoing[coeffs_outgoing["destination"].isin(countries_selected)]

In [22]:
trips_abroad_selected=trips_abroad[(trips_abroad["origin"].isin(countries_selected))|(trips_abroad["destination"].isin(countries_selected))]

In [23]:
trips_abroad_rescaled=rescale_trips(trips_abroad_selected,coeffs_incoming,coeffs_outgoing)

In [24]:
airport_codes=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\airports_info\IATA_ICAO_Airport_codes_v1.3.csv")
iata_to_icao=airport_codes.set_index("IATA")["ICAO"].to_dict()

In [25]:
#change the entry_point or exit_point to icao code
trips_abroad_rescaled["entry_point"]=trips_abroad_rescaled["entry_point"].apply(lambda x: format_airports(x,iata_to_icao))
trips_abroad_rescaled["exit_point"]=trips_abroad_rescaled["exit_point"].apply(lambda x: format_airports(x,iata_to_icao))

In [26]:
trips_going_to_spain=trips_abroad_rescaled[trips_abroad_rescaled["exit_point"].isna()]
trips_leaving_spain=trips_abroad_rescaled[trips_abroad_rescaled["entry_point"].isna()]

In [27]:
trips_going_to_spain

,date,origin,origin_name,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,legs,nationality,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,trips
182,20220923,AT,AT,ES111,A Coruña,LECO,NaN,NF,NF,P10*abroad_232*1503103*None*airport_LCG*00-01*...,AT,25.429960,19.375208,12.311330,2.220076,1.614601,1.412776,2,plane-road,airport_LCG,airport_LCG,airport_LCG,international_O,1,0,1,airport_LCG,airport_LCG,airport_LCG,['air'],62.363950
183,20220923,AT,AT,ES114,Pontevedra,LEVX,NaN,NF,NF,P19*abroad_232*3605707*None*airport_VGO*00-01*...,PL,4.672052,3.270437,1.635218,0.233603,0.467205,0.233603,2,plane-road,airport_VGO,airport_VGO,airport_VGO,international_O,1,0,1,airport_VGO,airport_VGO,airport_VGO,['air'],10.512118
184,20220923,AT,AT,ES120,Asturias,LEVT,NaN,NF,NF,P09*abroad_232*0105906*None*airport_VIT*00-01*...,AT,0.586863,0.365926,0.214032,0.069043,0.027617,0.041426,4,plane-road-bus-road,airport_VIT-bus_19-bus_2222,airport_VIT,bus_2222,international_O,2,0,1,airport_VIT-bus_19-bus_2222,airport_VIT,bus_2222,['air'],1.304907
185,20220923,AT,AT,ES120,Asturias,LEVT,NaN,NF,NF,P09*abroad_232*0105906*None*airport_VIT*00-01*...,AT,4.140809,2.581916,1.510177,0.487154,0.194862,0.292292,2,plane-road,airport_VIT,airport_VIT,airport_VIT,international_O,1,0,1,airport_VIT,airport_VIT,airport_VIT,['air'],9.207211
186,20220923,AT,AT,ES130,Cantabria,LEXJ,NaN,NF,NF,P09*abroad_232*3901601*None*airport_SDR*00-01*...,PL,3.997566,4.589798,1.332522,0.148058,0.148058,0.296116,2,plane-road,airport_SDR,airport_SDR,airport_SDR,international_O,1,0,1,airport_SDR,airport_SDR,airport_SDR,['air'],10.512118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72704,20220923,US,US,ES620,Murcia,LEAL,NaN,NF,NF,P21*abroad_310*0306507*None*airport_ALC*00-01*...,US,2.633276,0.957555,0.000000,0.000000,0.000000,0.000000,2,plane-road,airport_ALC,airport_ALC,airport_ALC,international_O,1,0,1,airport_ALC,airport_ALC,airport_ALC,['air'],3.590830
72705,20220923,US,US,ES620,Murcia,LEAL,NaN,NF,NF,P21*abroad_310*0306507*None*airport_ALC*00-01*...,US,2.633276,0.957555,0.000000,0.000000,0.000000,0.000000,2,plane-road,airport_ALC,airport_ALC,airport_ALC,international_O,1,0,1,airport_ALC,airport_ALC,airport_ALC,['air'],3.590830
72706,20220923,US,US,ES620,Murcia,LEAL,NaN,NF,NF,P21*abroad_311*0306507*None*airport_ALC*00-01*...,AR,2.633276,0.957555,0.000000,0.000000,0.000000,0.000000,2,plane-road,airport_ALC,airport_ALC,airport_ALC,international_O,1,0,1,airport_ALC,airport_ALC,airport_ALC,['air'],3.590830
72707,20220923,US,US,ES709,Tenerife,LEMD,NaN,NF,NF,P13*abroad_310*2807921*None*airport_MAD*00-01*...,US,0.542012,1.761539,0.880770,0.067752,0.203255,0.135503,3,plane-plane-road,airport_MAD-airport_TFN,airport_MAD,airport_TFN,international_O,1,0,2,airport_MAD-airport_TFN,airport_MAD,airport_TFN,"['air', 'air']",3.590830


Now I have trips going to spain and trips leaving spain and I have to assign them a path from MultiModX

## 2. Assign a path composed of MMX stations inside spain

(mmx_path)

In [28]:
# coordinates, geometry and other properties of all NUTS (in Europe?)
NUTS_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\nuts3_2003_geom_10.gpkg")

In [29]:
# list of stations considered in MMX
train_stations_considered=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\CS11\v=0.25\infrastructure\rail_info\rail_stations_considered_GTFS_2022v0.1.csv").astype(str)
train_stations_considered["stop_id"] = train_stations_considered["stop_id"].apply(lambda x: "00" + x) #to make they start with 00

In [30]:
# align coordinate reference system (crs)
NUTS_coord = NUTS_coord.to_crs(MobA_stations_coord.crs)

# Perform a spatial join to find which NUTS region each station belongs to
spatial_join = gpd.sjoin(MobA_stations_coord, NUTS_coord, how="left", predicate="within")

# Construct the dictionary. This dictionary tells us in which nuts is each station
station_to_nuts = dict(zip(spatial_join["ID"], spatial_join["geocode"]))

In [31]:
# adds a column for the moba acronym and another one with the nuts of the sation
train_stations_considered=format_train_stations_considered(train_stations_considered,station_to_nuts)

In [32]:
train_stations_considered

,stop_id,modified_id,NUTS
0,007102002,train_02002,ES613
1,007102003,train_02003,ES617
2,007102030,train_02030,ES617
3,007103100,train_03100,ES616
4,007103208,train_03208,ES423
...,...,...,...
85,007181202,train_81202,ES220
86,007182100,train_82100,ES417
87,007192102,train_92102,ES425
88,007167200,train_67200,ES242


In [33]:
# list of all MMX stations
train_station_MMX=train_stations_considered["stop_id"].tolist()

In [34]:
trips_going_to_spain=process_node_sequence_MMX(trips_going_to_spain,train_station_MMX,iata_to_icao)

In [35]:
trips_going_to_spain[trips_going_to_spain["node_sequence_MMX"].isna()]["trips"].sum()

np.float64(3909.243183540103)

In [36]:
trips_going_to_spain[trips_going_to_spain["node_sequence_MMX"].notna()]["trips"].sum()

np.float64(254894.22342886135)

In [37]:
trips_going_to_spain[(trips_going_to_spain["node_sequence_MMX"].notna())&(trips_going_to_spain["mode_tp"].str.contains("rail"))]["trips"].sum()

np.float64(18315.398471437715)

I only have very few incoming trips with no sequence (at least in spain) and I still have a lot of trips that contain rail and do have a node sequence

In [38]:
trips_leaving_spain=process_node_sequence_MMX(trips_leaving_spain,train_station_MMX,iata_to_icao)

In [39]:
trips_leaving_spain[trips_leaving_spain["node_sequence_MMX"].isna()]["trips"].sum()

np.float64(2026.190980000272)

In [40]:
trips_leaving_spain[trips_leaving_spain["node_sequence_MMX"].notna()]["trips"].sum()

np.float64(241694.80214576505)

In [41]:
trips_leaving_spain[(trips_leaving_spain["node_sequence_MMX"].notna())&(trips_leaving_spain["mode_tp"].str.contains("rail"))]["trips"].sum()

np.float64(11387.126316151473)

I have the same situation with trips leaving spain

## 3. Assign a path outside spain

I already decided that I will drop the archetypes

In [43]:
trips_leaving_spain=format_trips_leaving(trips_leaving_spain)

In [44]:
trips_going_to_spain=drop_archetypes(trips_going_to_spain)

In [45]:
trips_going_to_spain=trips_going_to_spain.rename(columns={"node_sequence_MMX":"node_sequence_spain"})

In [46]:
trips_leaving_spain=trips_leaving_spain.rename(columns={"node_sequence_MMX":"node_sequence_spain"})

In [51]:
departing_paths=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\CS11\aw\departing_paths.csv")

In [52]:
departing_paths.head()

,Unnamed: 0,exit,path,perc
0,0,GCFV,"['GCFV', 'EBBR']",0.003980
1,1,GCFV,"['GCFV', 'EBCI']",0.005183
2,2,GCFV,"['GCFV', 'EDDB']",0.009508
3,3,GCFV,"['GCFV', 'EDDC']",0.000226
4,4,GCFV,"['GCFV', 'EDDF']",0.053523


In [53]:
departing_paths=departing_paths.drop(["Unnamed: 0"],axis=1)

In [54]:
departing_paths["airport_nb"]=departing_paths["path"].apply(lambda x: len(ast.literal_eval(x)))

In [55]:
departing_paths[departing_paths["airport_nb"]==4]

,exit,path,perc,airport_nb
68,GCFV,"['GCLP', 'GCTS', 'GCFV', 'LIPE']",0.000091,4
259,GCLP,"['LEBL', 'GCXO', 'GCLP', 'LPPT']",0.000007,4
260,GCLP,"['LEIB', 'LEMD', 'GCLP', 'EIDW']",0.000009,4
261,GCLP,"['LEJR', 'LEMD', 'GCLP', 'GQPP']",0.000001,4
271,GCLP,"['LEMD', 'LEAS', 'GCLP', 'GOBD']",0.000010,4
...,...,...,...,...
6393,LEPA,"['LEMD', 'LEAL', 'LEPA', 'LFPO']",0.000002,4
6394,LEPA,"['LEMD', 'LEIB', 'LEPA', 'EGLL']",0.000005,4
6395,LEPA,"['LEMD', 'LEMH', 'LEPA', 'LFMN']",0.000003,4
6667,LEVC,"['LEIB', 'LEPA', 'LEVC', 'LROP']",0.000005,4


this phase was testing to see what would work

In [56]:
trips_leaving_spain[trips_leaving_spain["exit_point"]=="LECO"]

,origin,origin_name,exit_point,trips_from_exit_point,node_sequence_spain,trips
6,ES111,A Coruña,LECO,992.440801,['LECO'],884.756318
19,ES112,Lugo,LECO,992.440801,['LECO'],21.968543
81,ES212,Guipúzcoa,LECO,992.440801,['LECO'],0.744663
176,ES300,Madrid,LECO,992.440801,"['LEMD', 'LECO']",15.117680
488,ES617,Málaga,LECO,992.440801,"['LEMG', 'LEST', 'LECO']",13.342381
561,ES709,Tenerife,LECO,992.440801,"['GCXO', 'LEST', 'LECO']",56.511217


In [57]:
trips_leaving_leco=trips_leaving_spain[trips_leaving_spain["exit_point"]=="LECO"]

In [58]:
departing_paths_leco=departing_paths[departing_paths["exit"]=="LECO"]

In [59]:
departing_paths_leco

,exit,path,perc,airport_nb
2601,LECO,"['LECO', 'EDDH']",0.000624,2
2602,LECO,"['LECO', 'EGLL']",0.998746,2
2603,LECO,"['LECO', 'LIPE']",0.000630,2


In [60]:
np.random.seed(41)

In [61]:
trips_leaving_leco_w_paths=pd.DataFrame(columns=["origin","origin_name","node_sequence_spain","node_sequence_international","exit_point","trips"])
international_paths=departing_paths_leco["path"].values
probs=departing_paths_leco["perc"].values
probs=probs/probs.sum()

for idx, row in trips_leaving_leco.iterrows():
    origin=row["origin"]
    trips_count=row["trips"]
    origin_name=row["origin_name"]
    node_sequence=row["node_sequence_spain"]
    exit_point=row["exit_point"]
    for int in range(1,round(trips_count)+1):
        international_path_chosen=np.random.choice(international_paths,p=probs)
        trips_leaving_leco_w_paths.loc[trips_leaving_leco_w_paths.shape[0]]=[
            origin,
            origin_name,
            node_sequence,
            international_path_chosen,
            exit_point,
            float(1)
        ]
    trips_leaving_leco_w_paths=trips_leaving_leco_w_paths.groupby(["origin","origin_name","node_sequence_spain","node_sequence_international","exit_point"])["trips"].sum().reset_index()


In [63]:
# First, let's prepare the data
international_paths = departing_paths_leco["path"].values
probs = departing_paths_leco["perc"].values
probs = probs / probs.sum()  # Normalize probabilities

# Create an empty list to store all the rows
all_rows = []
trips_not_assigned=0

# Iterate through each origin
for idx, row in trips_leaving_leco.iterrows():
    origin = row["origin"]
    trips_count = round(row["trips"])  # Number of trips to generate
    origin_name = row["origin_name"]
    node_sequence = row["node_sequence_spain"]
    exit_point = row["exit_point"]

    if len(ast.literal_eval(node_sequence))==1:
        # Generate ALL path choices for this origin at once
        international_paths_chosen = np.random.choice(
            international_paths, 
            size=trips_count,  # Generate all needed samples in one call
            p=probs
        )
        
        # Create rows for all trips from this origin
        for path in international_paths_chosen:
            all_rows.append([
                origin,
                origin_name,
                node_sequence,
                path,
                exit_point,
                float(1)
            ])
    else:
        trips_not_assigned+=trips_count

# Create DataFrame from all rows at once (more efficient)
trips_leaving_leco_w_paths = pd.DataFrame(
    all_rows,
    columns=[
        "origin", "origin_name", "node_sequence_spain", 
        "node_sequence_international", "exit_point", "trips"
    ]
)

# Group by all relevant columns and sum trips
trips_leaving_leco_w_paths = trips_leaving_leco_w_paths.groupby([
    "origin", "origin_name", "node_sequence_spain", 
    "node_sequence_international", "exit_point"
])["trips"].sum().reset_index()

In [64]:
departing_paths.sample(10)

,exit,path,perc,airport_nb
706,LEBB,"['LEBB', 'LMML']",0.003416,2
353,GCTS,"['GCFV', 'GCTS', 'LIRF']",0.000026,3
2800,LEMD,"['GCFV', 'GCLP', 'LEMD', 'EDDF']",0.000010,4
1727,LEBL,"['LEIB', 'LEBL', 'LIEO']",0.000004,3
428,GCTS,"['GCTS', 'EPWR']",0.002931,2
3364,LEMD,"['GCXO', 'LEMD', 'EFHK']",0.000010,3
4830,LEMD,"['LEMD', 'OEJN']",0.001012,2
3903,LEMD,"['LEBB', 'LEMD', 'ELLX']",0.000004,3
32,GCFV,"['GCFV', 'EGPH']",0.020714,2
2768,LEIB,"['LEPA', 'LEIB', 'LFQQ']",0.000119,3


In [65]:
trips_leaving_leco_w_paths

,origin,origin_name,node_sequence_spain,node_sequence_international,exit_point,trips
0,ES111,A Coruña,['LECO'],"['LECO', 'EDDH']",LECO,2.0
1,ES111,A Coruña,['LECO'],"['LECO', 'EGLL']",LECO,882.0
2,ES111,A Coruña,['LECO'],"['LECO', 'LIPE']",LECO,1.0
3,ES112,Lugo,['LECO'],"['LECO', 'EGLL']",LECO,22.0
4,ES212,Guipúzcoa,['LECO'],"['LECO', 'EGLL']",LECO,1.0


In [66]:
trips_not_assigned

85

here is the implementation for trips leaving spain

In [146]:
trips_leaving_w_international_path=assign_international_path_leaving(trips_leaving_spain,departing_paths)

C:\Users\LMENENDEZ\GitHub\MultiModX\script\trips_format.py:991: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pass


No paths with exit point LEAS and spanish airports ['GCXO', 'LEAS']
56.511216876427405 were removed
No paths with exit point LEBB and spanish airports ['LEVX', 'LEMD', 'LEBB']
0.20173075810837843 were removed
No paths with exit point LEBB and spanish airports ['LEIB', 'LEBB']
0.7451286732841607 were removed
No paths with exit point LEBB and spanish airports ['LEMG', 'LEBB']
1.0387065745781077 were removed
No paths with exit point LEBB and spanish airports ['LEMH', 'LEBB', 'LEMD', 'LEBB']
0.7451286732841607 were removed
No paths with exit point LEBB and spanish airports ['LEBL', 'LEBB']
0.8705488218639299 were removed
No paths with exit point LEBB and spanish airports ['LEAL', 'LEBB']
2.027085508934396 were removed
No paths with exit point LEBB and spanish airports ['LEMH', 'LEBB']
0.13251769465696828 were removed
No paths with exit point LEBB and spanish airports ['GCXO', 'LEBL', 'LEBB']
0.2620811731910105 were removed
No paths with exit point LEGE and spanish airports ['LEIB', 'LEBL',

In [141]:
trips_leaving_w_international_path.sample(10)

,origin,origin_name,node_sequence_spain,node_sequence_international,exit_point,trips
4077,ES514,Tarragona,['LEBL'],"['LEBL', 'GMMW']",LEBL,4.0
2491,ES419,Zamora,['LEST'],"['LEST', 'EGSS']",LEST,2.0
3855,ES512,Girona,['LEGE'],"['LEGE', 'UUDD']",LEGE,24.0
7398,ES620,Murcia,['LEMD'],"['LEMD', 'KCLT']",LEMD,1.0
7570,ES707,La Palma,"['GCLA', 'LEMD']","['GCLA', 'LEMD', 'SUMU']",LEMD,1.0
5807,ES612,Cádiz,['LEMG'],"['LEMG', 'LPPR']",LEMG,1.0
6880,ES618,Sevilla,"['007151003', 'LEJR']","['LEJR', 'EDDH']",LEJR,13.0
7513,ES620,Murcia,['LEVC'],"['LEVC', 'GMFF']",LEVC,2.0
6730,ES617,Málaga,['LEMG'],"['LEMG', 'EHEH']",LEMG,563.0
3687,ES512,Girona,['LEBL'],"['LEBL', 'ENGM']",LEBL,14.0


In [142]:
trips_leaving_spain["trips"].sum()

np.float64(241694.80214576508)

In [143]:
trips_leaving_w_international_path["trips"].sum()

np.float64(295031.0)

the function assign_international_path creates a few new trips for o-d pairs that have less than 1 trip. This should not be a problem

In [118]:
departing_paths

,exit,path,perc,airport_nb,path_list
0,GCFV,"['GCFV', 'EBBR']",0.003980,2,"[GCFV, EBBR]"
1,GCFV,"['GCFV', 'EBCI']",0.005183,2,"[GCFV, EBCI]"
2,GCFV,"['GCFV', 'EDDB']",0.009508,2,"[GCFV, EDDB]"
3,GCFV,"['GCFV', 'EDDC']",0.000226,2,"[GCFV, EDDC]"
4,GCFV,"['GCFV', 'EDDF']",0.053523,2,"[GCFV, EDDF]"
...,...,...,...,...,...
6859,LEZL,"['LEZL', 'LPPT']",0.032797,2,"[LEZL, LPPT]"
6860,LEZL,"['LEZL', 'LSGG']",0.013402,2,"[LEZL, LSGG]"
6861,LEZL,"['LEZL', 'LSZH']",0.008663,2,"[LEZL, LSZH]"
6862,LEZL,"['LEZL', 'MMMX']",0.000013,2,"[LEZL, MMMX]"


In [92]:
trips_leaving_spain

,origin,origin_name,node_sequence_spain,exit_point,trips_from_exit_point,trips
0,ES111,A Coruña,"['007131400', '007117000', 'LEMD']",LEMD,46983.446044,1.455308
1,ES111,A Coruña,"['007131400', 'LEVX']",LEVX,2642.803771,8.820113
2,ES111,A Coruña,"['007131412', '007117000', 'LEMD']",LEMD,46983.446044,1.685976
3,ES111,A Coruña,['LEAS'],LEAS,474.519590,12.660660
4,ES111,A Coruña,['LEBB'],LEBB,8330.108851,0.202575
...,...,...,...,...,...,...
564,ES709,Tenerife,"['GCXO', 'LEBL']",LEBL,61563.559426,151.306205
565,ES709,Tenerife,"['GCXO', 'LEMD', 'LEVC', 'LEMD']",LEMD,46983.446044,7.505808
566,ES709,Tenerife,"['GCXO', 'LEMD']",LEMD,46983.446044,44.496536
567,ES709,Tenerife,"['GCXO', 'LEST', 'LECO']",LECO,992.440801,56.511217


## A. WIP

In [84]:
possible_itineraries_cp=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\CS11\v=0.25\output\processed_cs10.pp00.nd00.so00.00_c2\paths_itineraries\possible_itineraries_clustered_pareto_0.csv")

In [85]:
possible_itineraries_cp

,origin,destination,journey_type,cluster_id,alternative_id,options_in_cluster,total_travel_time,total_cost,total_emissions,total_waiting_time,nservices
0,EBBR,ES213,air,0,EBBR_ES213_0,"[0, 1]",156.00,176.55,87.09,0.00,1.0
1,EBBR,ES300,air,0,EBBR_ES300_0,"[0, 3, 4, 10, 11]",208.00,193.42,82.76,0.00,1.0
2,EBBR,ES300,air,1,EBBR_ES300_1,"[1, 2, 5, 6, 7, 8, 9]",206.57,193.42,109.33,0.00,1.0
3,EBBR,ES300,multimodal,12,EBBR_ES300_12,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21]",421.40,233.92,85.80,22.00,2.0
4,EBBR,ES415,air,0,EBBR_ES415_0,"[0, 3, 4, 10, 11]",379.00,193.42,82.76,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
10207,LSZH,ES617,multimodal,46,LSZH_ES617_46,[46],613.00,230.32,93.91,58.00,3.0
10208,LSZH,ES618,multimodal,0,LSZH_ES618_0,"[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15...",417.60,215.42,99.98,17.27,2.0
10209,LSZH,ES620,multimodal,0,LSZH_ES620_0,[0],487.00,227.56,129.64,5.00,2.0
10210,LSZH,ES620,multimodal,2,LSZH_ES620_2,[2],514.00,227.56,93.13,10.00,2.0


In [86]:
possible_paths_filtered=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\CS11\v=0.25\output\processed_cs10.pp00.nd00.so00.00_c2\paths_itineraries\possible_paths_avg_from_filtered_it_0.csv")

In [87]:
possible_paths_filtered

,path_id,n_itineraries,n_alternative_id,origin,destination,path,alternative_id,cluster_id,option,total_travel_time_min,total_travel_time_max,nservices,total_avg_travel_time,total_avg_cost,total_avg_emissions,earliest_departure_time,latest_arrival_time,total_avg_waiting_time,nmodes,journey_type,access_avg_time,egress_avg_time,d2i_time,i2d_time,origin_0,destination_0,mode_0,travel_avg_time_0,cost_avg_0,service_cost_0,emissions_cost_0,emissions_0,origin_1,destination_1,mode_1,travel_avg_time_1,mct_avg_time_0_1,ground_mobility_time_0_1,connecting_avg_time_0_1,waiting_avg_time_0_1,cost_avg_1,service_cost_1,emissions_cost_1,emissions_1,origin_2,destination_2,mode_2,travel_avg_time_2,mct_avg_time_1_2,ground_mobility_time_1_2,connecting_avg_time_1_2,waiting_avg_time_1_2,cost_avg_2,service_cost_2,emissions_cost_2,emissions_2,total_cost_min,total_emissions_min,total_waiting_time_min,travel_time_0_min,travel_time_1_min,travel_time_2_min,cost_0_min,cost_1_min,cost_2_min,connecting_time_0_1_min,connecting_time_1_2_min,waiting_time_0_1_min,waiting_time_1_2_min,total_cost_max,total_emissions_max,total_waiting_time_max,travel_time_0_max,travel_time_1_max,travel_time_2_max,cost_0_max,cost_1_max,cost_2_max,connecting_time_0_1_max,connecting_time_1_2_max,waiting_time_0_1_max,waiting_time_1_2_max
0,0,2,1,EBBR,ES213,"['EBBR', 'LEBB']",NaN,0.000000,0,156.0,156.0,1.0,156.000000,176.55,87.0900,2019-09-06 09:45:00+00:00,2019-09-06 11:40:00+00:00,0.0,1.0,NaN,0.0,41.0,0.0,11.0,EBBR,LEBB,air,115.000000,176.55,176.55,0.0,87.0900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,176.55,87.09,0.0,115.0,NaN,NaN,176.55,NaN,NaN,NaN,NaN,NaN,NaN,176.55,87.09,0.0,115.0,NaN,NaN,176.55,NaN,NaN,NaN,NaN,NaN,NaN
1,1,12,2,EBBR,ES300,"['EBBR', 'LEMD']",NaN,0.583333,0,203.0,213.0,1.0,207.166667,193.42,98.2575,2019-09-06 05:00:00+00:00,2019-09-06 08:00:00+00:00,0.0,1.0,NaN,0.0,63.0,0.0,33.0,EBBR,LEMD,air,144.166667,193.42,193.42,0.0,98.2575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193.42,80.36,0.0,140.0,NaN,NaN,193.42,NaN,NaN,NaN,NaN,NaN,NaN,193.42,80.84,0.0,150.0,NaN,NaN,193.42,NaN,NaN,NaN,NaN,NaN,NaN
2,2,10,1,EBBR,ES300,"['EBBR', 'LEBL', '007171801', '007160000']",NaN,12.000000,1,403.0,443.0,2.0,421.400000,233.92,85.7970,2019-09-06 04:40:00+00:00,2019-09-06 16:55:00+00:00,22.0,2.0,NaN,0.0,18.0,0.0,8.0,EBBR,LEBL,air,121.000000,180.96,180.96,0.0,71.2370,007171801,007160000,rail,155.40,105.0,60.0,127.0,22.0,52.96,52.96,0.0,14.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233.92,88.12,10.0,120.0,150.0,NaN,180.96,52.96,NaN,115.0,NaN,10.0,NaN,233.92,88.12,40.0,115.0,165.0,NaN,180.96,52.96,NaN,145.0,NaN,40.0,NaN
3,3,12,2,EBBR,ES415,"['EBBR', 'LEMD']",NaN,0.583333,0,374.0,384.0,1.0,378.166667,193.42,98.2575,2019-09-06 05:00:00+00:00,2019-09-06 08:00:00+00:00,0.0,1.0,NaN,0.0,234.0,0.0,204.0,EBBR,LEMD,air,144.166667,193.42,193.42,0.0,98.2575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193.42,80.36,0.0,140.0,NaN,NaN,193.42,NaN,NaN,NaN,NaN,NaN,NaN,193.42,80.84,0.0,150.0,NaN,NaN,193.42,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2,1,EBBR,ES415,"['EBBR', 'LEBL', '007171801', '007160000', '00...",NaN,12.000000,1,708.0,708.0,3.0,708.000000,250.11,93.1900,2019-09-06 04:40:00+00:00,2019-09-06 15:31:00+00:00,110.0,2.0,NaN,0.0,57.0,0.0,47.0,EBBR,LEBL,air,115.000000,180.96,180.96,0.0,73.5600,007171801,007160000,rail,165.00,105.0,60.0,135.0,30.0,52.96,52.96,0.0,14.56,007117000,007130100,rail,101.0,55.0,30.0,135.0,80.0,16.19,16.19,0.0,5.07,250.11,93.19,110.0,115.0,165.0,101.0,180.96,52.96,16.19,145.0,125.0,40.0,70.0,250.11,93.19,110.0,115.0,165.0,101.0,180.96,52.96,16.19,145.0,125.0,40.0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [88]:
iata_to_icao

{'AAC': 'HEAR',
 'AAE': 'DABB',
 'AAL': 'EKYT',
 'AAM': 'FAMD',
 'AAN': 'OMAL',
 'AAO': 'SVAN',
 'AAQ': 'URKA',
 'AAR': 'EKAH',
 'AAT': 'ZWAT',
 'AAV': 'RPMA',
 'AAY': 'OYGD',
 'AAZ': 'MGQZ',
 'ABC': 'LEAB',
 'ABD': 'OIAA',
 'ABE': 'KABE',
 'ABH': 'YAPH',
 'ABI': 'KABI',
 'ABJ': 'DIAP',
 'ABK': 'HAKD',
 'ABO': 'TJAB',
 'ABQ': 'KABQ',
 'ABR': 'KABR',
 'ABS': 'HEBL',
 'ABT': 'OEBA',
 'ABV': 'DNAA',
 'ABX': 'YMAY',
 'ABY': 'KABY',
 'ABZ': 'EGPD',
 'ACA': 'MMAA',
 'ACC': 'DGAA',
 'ACE': 'GCRR',
 'ACH': 'LSZR',
 'ACI': 'EGJA',
 'ACK': 'KACK',
 'ACR': 'SKAC',
 'ACT': 'KACT',
 'ACV': 'KACV',
 'ACY': 'KACY',
 'ACZ': 'OIZB',
 'ADA': 'LTAF',
 'ADB': 'LTBJ',
 'ADD': 'HAAB',
 'ADE': 'OYAA',
 'ADF': 'LTCP',
 'ADH': 'UEAA',
 'ADI': 'FYAR',
 'ADJ': 'OJAM',
 'ADL': 'YPAD',
 'ADP': 'VCCA',
 'ADQ': 'PADQ',
 'ADU': 'OITL',
 'AEH': 'FTTC',
 'AEO': 'GQNA',
 'AEP': 'SABE',
 'AES': 'ENAL',
 'AEX': 'KAEX',
 'AEY': 'BIAR',
 'AFA': 'SAMR',
 'AFD': 'FAPA',
 'AGA': 'GMAD',
 'AGB': 'EDMA',
 'AGE': 'EDWG',
 'AGF': 

In [89]:
airport_codes

,IATA,ICAO,Location,Airport,Country
0,AAC,HEAR,El Arish,El Arish Intl Airport,Egypt
1,AAE,DABB,Annaba,Annaba - El Mellah Airport,Algeria
2,AAL,EKYT,Aalborg,Aalborg Airport,Denmark
3,AAM,FAMD,Malamala,Malamala,South Africa
4,AAN,OMAL,Al Ain,Al Ain Intl,United Arab Emirates
...,...,...,...,...,...
4260,TCI,GCTS,Tenerife Generic,Tenefire South used as generic,Spain
4261,SCF,KSDL,Scttsdale,Scottdale Airport,USA
4262,PRC,KPRC,Prescott Regional,Prescott Regional Airport,USA
4263,NLC,KNLC,Leemoore,Lemoore,USA


In [90]:
airports_list=list(airport_codes["ICAO"])

In [91]:
spanish_airports=[airport for airport in airports_list if airport.startswith("LE") or airport.startswith("GC")]